In [ ]:
import torch

def negative_sampler_concept(self, data):

        with torch.no_grad():

            # Review this code
            
            # Sort the real batch by concept number and individual number
            print(f'data: {data}')
            print(f'data shape {data.shape}')

            sorted_concepts_idcs = torch.argsort(data[:,0])
            sorted_concepts_data = data[sorted_concepts_idcs]
            sorted_individual_idcs = torch.argsort(sorted_concepts_data[:,1] +
                                                   sorted_concepts_data[:,0] *
                                                   sorted_concepts_data.size(0))
            sorted_data_final = sorted_concepts_data[sorted_individual_idcs]

            print(f'sorted real batch conflict samples{sorted_data_final}')
            
            # Generate negative candidates and sort the corrupted batch

            neg_sampled_candidates = torch.randint(0, self.individual_embedding_dict.weight.shape[0], (data.shape[0],))
            corrupted_batch = torch.cat((data[:,0].unsqueeze(1), neg_sampled_candidates.unsqueeze(1)), dim=1)
            sorted_neg_concepts_idcs = sorted_concepts_idcs # Reutilize the previous sorting operation
            sorted_neg_concepts_batch = corrupted_batch[sorted_neg_concepts_idcs]
            sorted_corrupted_individual_idcs = torch.argsort(sorted_neg_concepts_batch[:,1] +
                                                             sorted_neg_concepts_batch[:,0] *
                                                             sorted_neg_concepts_batch.size(0))
            sorted_corrupted_batch_final = sorted_neg_concepts_batch[sorted_corrupted_individual_idcs]

            print(f'sorted corrupted conflict samples{sorted_corrupted_batch_final}')

            # While there are identical samples in both real and corrupted batches, generate more corrupted individuals

            # We check whether any assertion in our negatively sampled individuals is in the batch
            
            counter = 0
            MAX_ITER = 50000

            negsamp_checker = sorted_data_final[:,1] == sorted_corrupted_batch_final[:,1]

            while torch.any(negsamp_checker) and counter != MAX_ITER:
        
                conflict_idcs = torch.where(negsamp_checker == True)[0]
                for idx in conflict_idcs:
                    sorted_corrupted_batch_final[idx][1] = torch.randint(0, self.individual_embedding_dict.weight.shape[0], (1,)).item()

                counter += 1

            if counter == MAX_ITER:
                print('loop limit reached')
                #print(f'neg_sampchecker TRUE: {negsamp_checker}')
                where_conflict = torch.where(negsamp_checker)[0]

            # Return only the individuals, discarding concepts
            return sorted_corrupted_batch_final[:,1]